In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.7 MB/s e

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import precision_score, recall_score
import warnings

# Конфигурация
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 1  # Уменьшаем размер батча, так как изображений мало
NUM_CLASSES = 2
EPOCHS = 10

# Пути к данным
BASE_DIR = '/content/sample_data'
DIRS = {
    'train': ('train/images', 'train/masks'),
    'val': ('val/images', 'val/masks'),
    'test': ('test/images', 'test/masks')
}

# Функция для поиска пар изображение-маска
def find_image_mask_pairs(image_dir, mask_dir):
    pairs = []

    # Получаем все файлы в директориях
    image_files = {f.split('.')[0]: f for f in sorted(os.listdir(image_dir))
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))}
    mask_files = {f.split('.')[0]: f for f in sorted(os.listdir(mask_dir))
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))}

    # Находим общие имена без учета расширений
    common_names = set(image_files.keys()) & set(mask_files.keys())

    if not common_names:
        warnings.warn(f"Не найдено пар изображение-маска в {image_dir} и {mask_dir}")
        return pairs

    for name in common_names:
        img_path = os.path.join(image_dir, image_files[name])
        mask_path = os.path.join(mask_dir, mask_files[name])
        pairs.append((img_path, mask_path))

    return pairs

# Загружаем данные
data = {}
for split, (img_dir, mask_dir) in DIRS.items():
    full_img_dir = os.path.join(BASE_DIR, img_dir)
    full_mask_dir = os.path.join(BASE_DIR, mask_dir)

    if not os.path.exists(full_img_dir) or not os.path.exists(full_mask_dir):
        warnings.warn(f"Директории {full_img_dir} или {full_mask_dir} не существуют")
        data[split] = {'images': [], 'masks': []}
        continue

    pairs = find_image_mask_pairs(full_img_dir, full_mask_dir)
    if pairs:
        images, masks = zip(*pairs)
        data[split] = {'images': list(images), 'masks': list(masks)}
    else:
        data[split] = {'images': [], 'masks': []}

# Проверяем, есть ли данные для обучения
if not data['train']['images']:
    # Выводим содержимое директорий для отладки
    print("Содержимое train/images:", os.listdir(os.path.join(BASE_DIR, 'train/images')))
    print("Содержимое train/masks:", os.listdir(os.path.join(BASE_DIR, 'train/masks')))
    raise ValueError("Нет данных для обучения. Проверьте пути и файлы.")

# DataGenerator
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=1, image_size=(256, 256), shuffle=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index*self.batch_size:(index+1)*self.batch_size]
        batch_mask_paths = self.mask_paths[index*self.batch_size:(index+1)*self.batch_size]

        images = []
        masks = []

        for img_path, mask_path in zip(batch_image_paths, batch_mask_paths):
            # Загрузка изображения
            img = load_img(img_path, target_size=self.image_size)
            img = img_to_array(img) / 255.0
            images.append(img)

            # Загрузка маски
            mask = load_img(mask_path, target_size=self.image_size, color_mode='grayscale')
            mask = img_to_array(mask)
            mask = (mask > 128).astype(np.float32)  # Бинаризация маски
            masks.append(mask)

        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.image_paths))
            np.random.shuffle(indices)
            self.image_paths = [self.image_paths[i] for i in indices]
            self.mask_paths = [self.mask_paths[i] for i in indices]

# Создаем генераторы
train_generator = DataGenerator(data['train']['images'], data['train']['masks'], batch_size=BATCH_SIZE)
val_generator = DataGenerator(data['val']['images'], data['val']['masks'], batch_size=BATCH_SIZE) if data['val']['images'] else None
test_generator = DataGenerator(data['test']['images'], data['test']['masks'], batch_size=BATCH_SIZE, shuffle=False) if data['test']['images'] else None

# Упрощенная модель для малого количества данных
def simple_segmentation_model(input_size=(256, 256, 3)):
    inputs = tf.keras.Input(input_size)

    # Простая сверточная сеть
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)

    return models.Model(inputs, outputs)

model = simple_segmentation_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение
try:
    if val_generator:
        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE
        )
    else:
        history = model.fit(
            train_generator,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE
        )
except Exception as e:
    print(f"Ошибка при обучении: {e}")

# Оценка
if test_generator:
    try:
        # Функция для расчета метрик
        def calculate_metrics(model, generator):
            y_true = []
            y_pred = []

            for i in range(len(generator)):
                x, y = generator[i]
                pred = model.predict(x, verbose=0)

                y_true.extend(y.flatten())
                y_pred.extend((pred > 0.5).astype(np.float32).flatten())

            y_true = np.array(y_true)
            y_pred = np.array(y_pred)

            precision = precision_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)

            intersection = np.logical_and(y_true, y_pred)
            union = np.logical_or(y_true, y_pred)
            iou = np.sum(intersection) / np.sum(union)

            return precision, recall, iou

        precision, recall, iou = calculate_metrics(model, test_generator)
        print(f"\nРезультаты на тестовых данных:")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"IoU: {iou:.4f}")

    except Exception as e:
        print(f"Ошибка при оценке: {e}")

# Сохранение модели
try:
    model.save('segmentation_model.h5')
    print("Модель успешно сохранена")
except Exception as e:
    print(f"Ошибка при сохранении модели: {e}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 603ms/step - accuracy: 0.4311 - loss: 0.6953 - val_accuracy: 0.7188 - val_loss: 0.6265
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.6891 - loss: 0.6063 - val_accuracy: 0.7188 - val_loss: 0.5864
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 365ms/step - accuracy: 0.6891 - loss: 0.5671 - val_accuracy: 0.7188 - val_loss: 0.5622
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 353ms/step - accuracy: 0.6903 - loss: 0.5445 - val_accuracy: 0.7188 - val_loss: 0.5588
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 377ms/step - accuracy: 0.7247 - loss: 0.5021 - val_accuracy: 0.7188 - val_loss: 0.5656
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 414ms/step - accuracy: 0.6903 - loss: 0.5549 - val_accuracy: 0.7188 - val_loss: 0.5610
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 610ms/step - accuracy: 0.6903 - loss: 0.5465 - val_accuracy: 0.7188 - val_loss: 0.5461
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/step - accuracy: 0.6903 - loss: 0.5263 - val_accuracy: 0.7188 - val_loss:


Результаты на тестовых данных:
Precision: 1.0000
Recall: 0.0034
IoU: 0.0034
Модель успешно сохранена
